In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [7]:
df = pd.read_csv("../csv_outputs/master_liquidity.csv")

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2123 entries, 0 to 2122
Data columns (total 19 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   token_address                  2123 non-null   object 
 1   chain                          2123 non-null   object 
 2   token_name                     2123 non-null   object 
 3   price_usd                      2065 non-null   float64
 4   liquidity_usd                  2065 non-null   float64
 5   market_cap_usd                 2065 non-null   float64
 6   liquidity_to_market_cap_ratio  2065 non-null   float64
 7   token_volume                   2065 non-null   float64
 8   volume_usd                     2065 non-null   float64
 9   volume_to_liquidity_ratio      2065 non-null   float64
 10  locked_liquidity_percent       2065 non-null   float64
 11  locked_95_for_15_days          2065 non-null   object 
 12  creator_under_5_percent        2065 non-null   o